In [1]:
lines = []
import json
import ast
with open("RC/cornell-movie-dialogs-corpus/movie_conversations.txt", buffering=1000, encoding='utf8', errors='ignore') as f:
    for row in f:
        line = row.split(" +++$+++ ")
        x = ast.literal_eval(line[3].replace("\n",""))
        lines.append(x)
lines[:10]

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208'],
 ['L271', 'L272', 'L273', 'L274', 'L275'],
 ['L276', 'L277'],
 ['L280', 'L281'],
 ['L363', 'L364'],
 ['L365', 'L366']]

In [4]:
import unicodedata, re
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())w = re.sub(r'\.\.+',".",w)
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.rstrip().strip()

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

In [2]:
data = {}
with open("RC/cornell-movie-dialogs-corpus/movie_lines.txt", "r", encoding='utf-8', errors='ignore') as f:
    for row in f:
        try:
            
            line = row.split(" +++$+++ ")
#             data[line[0]] = preprocess_sentence(line[4].replace("\n",""))
            data[line[0]] = line[4].replace("\n","")
        except Exception as e:
            print(row)
data

{'L1045': 'They do not!',
 'L1044': 'They do to!',
 'L985': 'I hope so.',
 'L984': 'She okay?',
 'L925': "Let's go.",
 'L924': 'Wow',
 'L872': "Okay -- you're gonna need to learn how to lie.",
 'L871': 'No',
 'L870': 'I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869': 'Like my fear of wearing pastels?',
 'L868': 'The "real you".',
 'L867': 'What good stuff?',
 'L866': "I figured you'd get to the good stuff eventually.",
 'L865': 'Thank God!  If I had to hear one more story about your coiffure...',
 'L864': "Me.  This endless ...blonde babble. I'm like, boring myself.",
 'L863': 'What crap?',
 'L862': 'do you listen to this crap?',
 'L861': 'No...',
 'L860': 'Then Guillermo says, "If you go any lighter, you\'re gonna look like an extra on 90210."',
 'L699': 'You always been this selfish?',
 'L698': 'But',
 'L697': "Then that's all you had to say.",
 'L696': 'Well, no...',
 'L695': "You never wanted to go out with 'me, did y

In [3]:
qna = []
for line in lines:
    for i in range(len(line)-1):
        qna.append([data[line[i]], data[line[i+1]]])
qna

[['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.',
  "Well, I thought we'd start with pronunciation, if that's okay with you."],
 ["Well, I thought we'd start with pronunciation, if that's okay with you.",
  'Not the hacking and gagging and spitting part.  Please.'],
 ['Not the hacking and gagging and spitting part.  Please.',
  "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?"],
 ["You're asking me out.  That's so cute. What's your name again?",
  'Forget it.'],
 ["No, no, it's my fault -- we didn't have a proper introduction ---",
  'Cameron.'],
 ['Cameron.',
  "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does."],
 ["The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.",
  'Seems like she could get a date easy enough...'],
 [

In [4]:
import pandas as pd
pd.options.display.max_colwidth = 500
question = []
answer = []
for _ in qna:
    question.append(_[0])
    answer.append(_[1])
len(question), len(answer)

(221616, 221616)

In [5]:
df = pd.DataFrame(list(zip(question,answer)), columns=['questions','answers'])
df.head()

,questions,answers
0,Can we make this quick? Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad. Again.,"Well, I thought we'd start with pronunciation, if that's okay with you."
1,"Well, I thought we'd start with pronunciation, if that's okay with you.",Not the hacking and gagging and spitting part. Please.
2,Not the hacking and gagging and spitting part. Please.,Okay... then how 'bout we try out some French cuisine. Saturday? Night?
3,You're asking me out. That's so cute. What's your name again?,Forget it.
4,"No, no, it's my fault -- we didn't have a proper introduction ---",Cameron.


In [11]:
questions = df['questions'].values
answers = df['answers'].values
for i in range(len(questions)):
    if i < 220000:
        with open("train.from", 'a') as f:
            f.writelines(questions[i]+"\n")
        with open("train.to",'a') as z:
            z.writelines(answers[i]+"\n")
    else:
        with open("test.from", 'a') as f:
            f.writelines(questions[i]+"\n")
        with open("test.to",'a') as z:
            z.writelines(answers[i]+"\n")

In [13]:
with open("train.from",'r') as f:
    x = f.readlines()
print(len(x))

220000


In [15]:
df.to_csv("qna.csv")

In [ ]:
for i,_ in enumerate(z):
    x = _.split("'")
    if x[-1] == 's':
        z[i] = x[0]+' is'
    if x[-1] == 're':
        z[i] = x[0]+' are'
    if x[-1] == 't':
        z[i] = x[0][:-1]+ " not"
    if x[-1] == 'd':
        z[i] == x[0]+ " would"
    if x[-1] == 'll':
        z[i] == x[0]+ " will"
    if x[-1] == 've':
        z[i] == x[0]+ " have"
z

In [14]:
import sqlite3
sql_transaction = []
conn = sqlite3.connect("chatbot/db/RC_2018-09.db")
c = conn.cursor()

def transaction_bldr(sql):
    global sql_transaction
    sql_transaction.append(sql)
    if len(sql_transaction) > 1000:
        print("begin transcation",sql_transaction)
        c.execute('BEGIN TRANSACTION')
        for s in sql_transaction:
            try:
                c.execute(s)
            except:
                pass
        conn.commit()
        sql_transaction = []
#conn.close()

In [19]:
# c.execute("""SELECT count(*) FROM parent_reply""")
c.execute("""SELECT count(*) FROM parent_reply LIMIT 10""")
result = c.fetchall()
result
#sql_transaction

[(6115827,)]

In [20]:
# def create_table():
#     c.execute("CREATE TABLE IF NOT EXISTS test1(parent_id TEXT PRIMARY KEY, comment_id TEXT UNIQUE, parent TEXT, comment TEXT, subreddit TEXT, unix INT, score INT)")

# create_table()    
# c.execute("""INSERT INTO test1 (parent_id, parent, comment) VALUES ("{}","{}","{}");""".format("0001", "how are you ?","i am fine , thanks"))
# c.execute("SELECT * FROM test1 LIMIT 100")
# result = c.fetchall()
# result
# c.execute("""INSERT INTO parent_reply (parent_id, parent, comment) VALUES ("{}","{}","{}");""".format("0001", "how are you ?","i am fine , thanks"))
conn.commit()

In [18]:
for i,j in enumerate(qna):
#     print(str(i), j[0], j[1])
    if i != 0:
        sql = """INSERT INTO parent_reply (parent_id, parent, comment) VALUES ('{}', '{}', '{}');""".format(str(i), j[0], j[1])
        c.execute(sql)

## PREPARE DATASET FOR TRAINING AND TESTING

In [52]:
import sqlite3
import pandas as pd
import os
db = "chatbot/db/RC_2018-09.db"
datapath = "chatbot/data"
connection = sqlite3.connect(db)
cursor = connection.cursor()
limit = 10000
cur_length = limit
last_time = 0
counter = 0
test_counter = 0
train_counter = 0
while cur_length == limit:
    counter+= 1
    df = pd.read_sql("SELECT parent, comment, unix FROM parent_reply WHERE parent NOT NULL AND unix > {} AND score > 0 ORDER BY unix ASC LIMIT {}".format(last_time, limit), connection)
    cur_length = len(df)
    last_time = df["unix"].tail(1).values[0]
    if counter % 20 == 0:
        test_counter+= 1
        with open(os.path.join(datapath, "test.from1"),'a', encoding='utf8') as f:
            for content in df['parent'].values:
                f.write(str(content) + '\n')
        with open(os.path.join(datapath, "test.to1"),'a', encoding='utf8') as f:
            for content in df['comment'].values:
                f.write(str(content) + '\n')
        print("{} rows placed in test files".format(test_counter*limit))
    else:
        train_counter+= 1
        with open(os.path.join(datapath, "train.from"),'a', encoding='utf8') as f:
            for content in df['parent'].values:
                f.write(str(content) + "\n")
        with open(os.path.join(datapath, "train.to1"),'a', encoding='utf8') as f:
            for content in df['comment'].values:
                f.write(str(content) + "\n")
                break
        print("{} rows placed in train files".format(train_counter*limit))

10000 rows placed in train files
20000 rows placed in train files


KeyboardInterrupt: 

In [28]:
test = 

In [46]:
count = 0
rowto = []
with open(datapath+"/test.to") as f:
    for row in f:
        count+= 1
        rowto.append(row)
        if count == 20:
            break
rowto

['<start> this sub was started because r cringe refused to allow low hanging fruit like otherkin and other tumblr related nonsense <end>\n',
 '<start> okay ra s al ghul . <end>\n',
 '<start> agreed . one man taking on everyone else and trying to rip them all down . <end>\n',
 '<start> illinoid tbh <end>\n',
 '<start> absolutely agree with you <end>\n',
 '<start> lake coeur d alane <end>\n',
 '<start> kalimdor is both the name of the current continent and the giant landmass of old . they share a name . <end>\n',
 '<start> you re one of the few ive seen who has mentioned good tracking . i bit the bullet and got the game recently . though i love it so far , myself and others on my team often note drift and weird tracking . farpoint and arizona are all spot on with the aim , but for some reason firewall isn t . i feel i have a pretty optimal camera setup . . . feet high , angled down , but the gun is always in an odd position during matches . <end>\n',
 '<start> there are those who underst

In [47]:
[_ for _ in zip(rows, rowto)]

[('<start> it definitely wasn t always like this <end>\n',
  '<start> this sub was started because r cringe refused to allow low hanging fruit like otherkin and other tumblr related nonsense <end>\n'),
 ('<start> and that kids , is the story of how i accidentally burned down an entire city . <end>\n',
  '<start> okay ra s al ghul . <end>\n'),
 ('<start> for any bitcoin cash supporters reading this , take a moment and think critically . craig disagreed with where the entire bitcoin cash community roger ver , haipo , jihan , rick falkvinge , jonald fyookball , abc , bu , etc is going . even if you re a blind craig supporter , i hope there is some sense left in you to think for a moment . is it more likely that the problem lies on a single person or the entire bch community ? if craig wants to make bch the global p p money , i am all up for it . but if he wants to split destroy the community and start smearing anyone who disagrees with him , then i simply cannot see myself aligned with su

In [2]:
['10','2'].index('2')

1